1/ Importation des données TC et TITMC 

2/ Import et traitement du fichier des prénoms 

3/ Ajouter le genre des personnes 

4/ Codification des adresses 

5/ Ajout et traitement du code commune

6/ Retraiter et simplifier les qualités 

# Lecture de la base inpi
